# Importing libraries

In [1]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from time import sleep
import pandas as pd

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

# Functions

In [2]:
def get_prices(flight):
    class_price = '"flight-container"'
    button_price = flight.find_element_by_xpath(f'.//div[@class={class_price}]/button')
    button_price.click()
    #flight.click()
    #classes names to get fares
    class_fares_container = '"fares-table-container"'
    class_td = '"fare-"'
    
    #get fares objects
    fares = flight.find_elements_by_xpath(f'//div[@class = {class_fares_container}]//tfoot//td[contains(@class, {class_td})]')
    fares = [{'fare_object': fare} for fare in fares]
    
    #Classes names to get the fares data
    class_currency ='"currency-symbol"' 
    class_value = '"value"'
    class_name = '"radio"'
    
    #Get fares data
    for fare in fares:
        fare['name'] = fare['fare_object'].find_element_by_xpath(f'.//div[@class = {class_name}]/label').get_attribute('for')
        fare['currency'] = fare['fare_object'].find_element_by_xpath(f'.//span[@class = {class_currency}]').text
        fare['value'] = fare['fare_object'].find_element_by_xpath(f'.//span[@class = {class_value}]').text    
    for fare in fares:
        del fare['fare_object']
    button_price.click()

    return fares
#print(get_prices(flight))

In [3]:
def get_stops(flight):
    ##finds the stops button
    class_flight_stops = '"flight-summary-stops-description"'
    button_stops = flight.find_element_by_xpath(f'.//div[@class={class_flight_stops}]/button')
    button_stops.click()
    ##gets the stops and connections
    
    #classes names
    class_segments = '"sc-hZSUBg gfeULV"'
    class_each_segment = '"sc-cLQEGU hyoued"'
    class_connections = '"sc-cLQEGU dnKRNG"'
    
    #finds stops and connections
    stops = flight.find_elements_by_xpath(f'//div[@class={class_segments}]/div[@class={class_each_segment}]')
    connections = flight.find_elements_by_xpath(f'//div[@class={class_segments}]/div[@class={class_connections}]')    


    ## Gets stops information
    #class names
    class_departure = '"sc-bwCtUz iybVbT"'
    class_arrival = '"sc-bwCtUz iybVbT"'
    class_flight_duration = '"sc-esjQYD dMquDU"'
    class_scale_duration = '"sc-esjQYD dMquDU"'
    class_airline_details = '"airline-flight-details"'
    class_plane_number = '"sr-only"'
    class_plane_model = '"sc-gzOgki uTyOl"'

    #Adding stops into a dictionary
    stops_data = {}
    stops_data = [{'stop_object': stop} for stop in stops]

    #Adding connections into the dictionary
    for n in range(len(stops)):
        try:
            stops_data[n]['connection_object'] = connections[n]
        except:
             stops_data[n]['connection_object'] = None
    #Adding stops information into the dictionary
    for stop in stops_data:
        stop['departure_city'] = stop['stop_object'].find_element_by_xpath(f'.//div[@class={class_departure}]/abbr').text
        stop['departure_time'] = stop['stop_object'].find_element_by_xpath(f'.//div[@class={class_departure}]/time').get_attribute('datetime')
        stop['arrival_city'] = stop['stop_object'].find_element_by_xpath(f'.//div[@class={class_arrival}]/abbr').text
        stop['arrival_time'] = stop['stop_object'].find_element_by_xpath(f'.//div[@class={class_arrival}]/time').get_attribute('datetime')
        stop['flight_duration'] = stop['stop_object'].find_element_by_xpath(f'.//span[@class={class_flight_duration}]/time').get_attribute('datetime')
        stop['plane_number'] = stop['stop_object'].find_element_by_xpath(f'.//div[@class={class_airline_details}]/span[@class={class_plane_number}]').text
        stop['plane_model'] = stop['stop_object'].find_element_by_xpath(f'.//div[@class={class_airline_details}]/span[@class={class_plane_model}]').text
        try:
            stop['scale_duration'] = stop['connection_object'].find_element_by_xpath(f'.//span[@class={class_scale_duration}]/time').get_attribute('datetime')
        except:
            stop['scale_duration']= '0:00'
        #deletes stop and connection object
        del stop['stop_object']
        del stop['connection_object']
    try:
        driverChrome.find_element_by_xpath('//button[@class="close"]').click()
    except:
        pass
    return stops_data
#print(get_stops(flight))

In [4]:
def get_timing(flight):
    departure_time = flight.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
    arrival_time = flight.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
    duration = flight.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')
    return {'departure_time': departure_time, 'arrival_time':arrival_time,duration:'duration'}

In [5]:
def get_info(driver):
    button_close = driver.find_element_by_xpath('//button[@class="close"]')
    button_close.click()
    
    sleep(3)
    flights = driver.find_elements_by_xpath('//li[@class="flight"]')
    n = 1
    print(f'{len(flights)} found, starting to scrap')
    info = []
    for flight in flights:
        print(f'\n***Getting information of flight {n}***')
        times = get_timing(flight)
        prices = get_prices(flight)
        stops = get_stops(flight)
        info.append({'times': times, 'prices':prices, 'stops':stops})
        n+=1
    return info


# Main

In [6]:
url = "https://www.latam.com/en_un/apps/personas/booking?fecha1_dia=30&fecha1_anomes=2020-06&fecha2_dia=31&fecha2_anomes=2020-07&from_city2=SAO&to_city2=MEX&auAvailability=1&ida_vuelta=ida_vuelta&vuelos_origen=Mexico%20City&from_city1=MEX&vuelos_destino=Sao%20Paulo&to_city1=SAO&flex=1&vuelos_fecha_salida_ddmmaaaa=07/05/2020&vuelos_fecha_regreso_ddmmaaaa=09/06/2020&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo=&stopover_outbound_days=0&stopover_inbound_days=0&application=#/"
class_flight = '"flight"'

options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driverChrome = webdriver.Chrome(executable_path='./chromedriver', options=options)
driverChrome.get(url)

#Dynamic delay
delay = 10
try: 
    #adding delay
    flight = WebDriverWait(driverChrome,delay).until(EC.presence_of_element_located((By.XPATH, f'//li[@class = {class_flight}]')))
    print('Page loaded')
    flights_info = get_info(driverChrome)
except TimeoutException:
    print('Timeout')
    flights_info = []
driverChrome.close()
print(flights_info)

Page loaded
11 found, starting to scrap

***Getting information of flight 1***

***Getting information of flight 2***

***Getting information of flight 3***

***Getting information of flight 4***

***Getting information of flight 5***

***Getting information of flight 6***

***Getting information of flight 7***

***Getting information of flight 8***

***Getting information of flight 9***

***Getting information of flight 10***

***Getting information of flight 11***
[{'times': {'departure_time': '18:00', 'arrival_time': '05:20', 'PT9H20M': 'duration'}, 'prices': [{'name': 'PROMO', 'currency': 'US$', 'value': '264'}, {'name': 'LIGHT', 'currency': 'US$', 'value': '274'}, {'name': 'PLUS', 'currency': 'US$', 'value': '305'}, {'name': 'TOP', 'currency': 'US$', 'value': '1,452'}], 'stops': [{'departure_city': 'MEX', 'departure_time': '18:00', 'arrival_city': 'MEX', 'arrival_time': '18:00', 'flight_duration': '9:20', 'plane_number': 'Flight', 'plane_model': 'Airbus 350-900', 'scale_duration':

# << Draft >>

In [7]:
#response = requests.get(url)
#response.status_code

In [8]:
#s = bs(response.text, 'lxml')

In [9]:
#sleep(5)
#button_close = driverChrome.find_element_by_xpath('//button[@class="close"]')
#button_close.click()


### Selecting elements

In [10]:
#sleep(3)
#flights = driverChrome.find_elements_by_xpath('//li[@class="flight"]')

# #xpath version
# flight = flights[-1]
# departure_time = flight.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
# arrival_time = flight.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
# duration = flight.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')
# print(departure_time)
# print(arrival_time)
# print(duration)

# #CSS selectors version
# flight2 = flights[0] 
# flight2_data = {}
# flight2_data['departure'] = flight2.find_element_by_css_selector('.departure > time').get_attribute('datetime')
# flight2_data['arrival'] = flight2.find_element_by_css_selector('.arrival > time').get_attribute('datetime')
# flight2_data['duration'] = flight2.find_element_by_css_selector('.duration > time').get_attribute('datetime')
# print(flight2_data)



### Interacting with selectors
#### Getting stops

In [11]:
# #finds the stops button
# class_flight_stops = '"flight-summary-stops-description"'
# button_scales = flight.find_element_by_xpath(f'.//div[@class={class_flight_stops}]/button')
# button_scales.click()


In [12]:
# #gets the stops and connections
# class_segments = '"sc-hZSUBg gfeULV"'
# class_each_segment = '"sc-cLQEGU hyoued"'
# class_connections = '"sc-cLQEGU dnKRNG"'
# stops = flight.find_elements_by_xpath(f'//div[@class={class_segments}]/div[@class={class_each_segment}]')
# connections = flight.find_elements_by_xpath(f'//div[@class={class_segments}]/div[@class={class_connections}]')
# stops_number = len(connections)
# stops_number

In [13]:
# ## Gets stops information
# #class names
# class_departure = '"sc-bwCtUz iybVbT"'
# class_arrival = '"sc-bwCtUz iybVbT"'
# class_flight_duration = '"sc-esjQYD dMquDU"'
# class_scale_duration = '"sc-esjQYD dMquDU"'
# class_airline_details = '"airline-flight-details"'
# class_plane_number = '"sr-only"'
# class_plane_model = '"sc-gzOgki uTyOl"'

# #Adding stops into a dictionary
# stops_data = {}
# stops_data = [{'stop_object': stop} for stop in stops]

# #Adding connections into the dictionary
# for n in range(len(stops)):
#     try:
#         stops_data[n]['connection_object'] = connections[n]
#     except:
#          stops_data[n]['connection_object'] = None
# #Adding stops information into the dictionary
# for stop in stops_data:
#     stop['departure_city'] = stop['stop_object'].find_element_by_xpath(f'.//div[@class={class_departure}]/abbr').text
#     stop['departure_time'] = stop['stop_object'].find_element_by_xpath(f'.//div[@class={class_departure}]/time').get_attribute('datetime')
#     stop['arrival_city'] = stop['stop_object'].find_element_by_xpath(f'.//div[@class={class_arrival}]/abbr').text
#     stop['arrival_time'] = stop['stop_object'].find_element_by_xpath(f'.//div[@class={class_arrival}]/time').get_attribute('datetime')
#     stop['flight_duration'] = stop['stop_object'].find_element_by_xpath(f'.//span[@class={class_flight_duration}]/time').get_attribute('datetime')
#     stop['plane_number'] = stop['stop_object'].find_element_by_xpath(f'.//div[@class={class_airline_details}]/span[@class={class_plane_number}]').text
#     stop['plane_model'] = stop['stop_object'].find_element_by_xpath(f'.//div[@class={class_airline_details}]/span[@class={class_plane_model}]').text
#     try:
#         stop['scale_duration'] = stop['connection_object'].find_element_by_xpath(f'.//span[@class={class_scale_duration}]/time').get_attribute('datetime')
#     except:
#         stop['scale_duration']= '0:00'
#     print(stop)

In [14]:
# driverChrome.find_element_by_xpath('//button[@class="close"]').click()


In [15]:
# class_fares_container = '"fares-table-container"'
# class_td = '"fare-"'
# fares = flight.find_elements_by_xpath(f'//div[@class = {class_fares_container}]//tfoot//td[contains(@class, {class_td})]')
# fares = [{'fare_object': fare} for fare in fares]

# class_currency ='"currency-symbol"' 
# class_value = '"value"'
# class_name = '"radio"'
# for fare in fares:
#     fare['name'] = fare['fare_object'].find_element_by_xpath(f'.//div[@class = {class_name}]/label').get_attribute('for')
#     fare['currency'] = fare['fare_object'].find_element_by_xpath(f'.//span[@class = {class_currency}]').text
#     fare['value'] = fare['fare_object'].find_element_by_xpath(f'.//span[@class = {class_value}]').text
# fares

In [16]:
# for flight in flights:
#     print(flight)
#     print(get_timing(flight))
#     print(get_prices(flight))
#     print(get_stops(flight))

#    print('*************************************')